In [1]:
import sys
sys.path.append('..')

from collections import defaultdict

import joblib
import pickle
from tqdm.auto import tqdm

from github_utils import get_merge_commit_sha, get_python_files_from_pull_request
from git_diff_utils import get_function_versions

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('mode.sim_interactive', True)
pd.set_option('expand_frame_repr', False)
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 1000)

In [36]:
issues = pd.read_csv('../../data/3_repo_issues.csv')

In [37]:
issues.url.apply(lambda x: re.findall('(.+)\/\d+$', x)[0]).value_counts()

https://github.com/pandas-dev/pandas/issues            3656
https://github.com/numpy/numpy/issues                  823 
https://github.com/scikit-learn/scikit-learn/issues    778 
Name: url, dtype: int64

In [40]:
issues['repo_name'] = issues.url.apply(lambda x: x.split('/')[3] + '/' + x.split('/')[4])

In [41]:
issues['repo_name'].value_counts()

pandas-dev/pandas            3656
numpy/numpy                  823 
scikit-learn/scikit-learn    778 
Name: repo_name, dtype: int64

In [42]:
repo_name2path = {
    'scikit-learn/scikit-learn': '/Users/ilyamezentsev/Documents/zephyr/repos/scikit-learn/',
    'pandas-dev/pandas': '/Users/ilyamezentsev/Documents/zephyr/repos/pandas/',
    'numpy/numpy': '/Users/ilyamezentsev/Documents/zephyr/repos/numpy/'
}

In [45]:
def get_rows_for_prs(issue, column_name, repo_path):
    _file_versions_df = pd.DataFrame(columns=[
        'issue_url',
        'pr_type',
        'pr_url', 
        'merge_commit_sha', 
        'py_filename',
        'function_name',
        'before_merge',
        'after_merge'
    ])

    prs = eval(issue[column_name])
    if len(prs) != 0:
        for pr in prs:
            pr_url = pr['url']
            is_merged = pr['merged']

            if not is_merged:
                not_merged_prs.append(pr_url)
                continue

            if 'mergeCommit' not in pr.keys() or not pr['mergeCommit']:
                prs_without_merge_commit_urls.append(pr_url)
                continue
            
            merge_commit_sha = pr['mergeCommit']['commitUrl'].split('/')[-1]
            func_versions, _, _ = get_function_versions(
                repo_path=repo_path,
                commit_sha=merge_commit_sha,
                files=None
            )
            
            if not func_versions:
                    continue
                
            for filename, functions in func_versions.items():
                if len(functions) == 0:
                    prs_without_changed_functions.append(pr_url)
                    continue
                for function_name, versions in functions.items():
                    _file_versions_df = _file_versions_df.append([{
                        'issue_url': issue['url'], 
                        'pr_type': column_name,
                        'pr_url': pr_url,
                        'merge_commit_sha': merge_commit_sha,
                        'filename': filename,
                        'function_name': function_name,
                        'before_merge': versions['old'],
                        'after_merge': versions['new']
                    }])
    return _file_versions_df

In [4]:
sklearn_issues = pd.read_csv('../../data/scikit-learn_scikit-learn_issues.csv')

In [46]:
def get_rows_for_commits(issue, column_name, repo_path):
    _file_versions_df = pd.DataFrame(columns=[
        'issue_url',
        'commit_type',
        'commit_sha', 
        'py_filename',
        'function_name',
        'before_merge',
        'after_merge'
    ])

    commits = eval(issue[column_name])
    if len(commits) == 0:
        return
    for commit in commits:
        commit_sha = commit['commitUrl'].split('/')[-1]
        func_versions, _, _ = get_function_versions(
                repo_path=repo_path,
                commit_sha=commit_sha,
                files=None
        )

        if not func_versions:
            continue
                
        for filename, functions in func_versions.items():
            if len(functions) == 0:
                continue
            for function_name, versions in functions.items():
                _file_versions_df = _file_versions_df.append([{
                    'issue_url': issue['url'], 
                    'commit_type': column_name,
                    'commit_sha': commit_sha,
                    'filename': filename,
                    'function_name': function_name,
                    'before_merge': versions['old'],
                    'after_merge': versions['new']
                }])
    return _file_versions_df

In [48]:
not_merged_prs = []
prs_without_merge_commit_urls = []
prs_without_py_files = []
prs_without_changed_functions = []

file_versions_df = pd.DataFrame(columns=[
    'issue_url',
    'merge_commit_sha', 
    'filename',
    'function_name',
    'before_merge',
    'after_merge'
])

# repo_name = 'scikit-learn/scikit-learn'
# repo_path = '/Users/ilyamezentsev/Documents/zephyr/repos/scikit-learn/'

for index, issue in tqdm(issues.sort_values(by='closedAt', ascending=False).iterrows(), 
                         total=len(issues)):
    repo_name = issue.repo_name
    repo_path = repo_name2path[repo_name]
    for pr_column_name in ['closing PRs', 
                           'linked PRs',
                           'mentioning PRs',
                           'PRs for referencing commits']:
        _file_versions_df = get_rows_for_prs(issue, pr_column_name, repo_path)
        file_versions_df = file_versions_df.append(_file_versions_df)
        
    for commit_column_name in ['referencing commits not linked to PRs', 
                               'referencing commits linked to PRs',
                               'closing commits not linked to PRs',
                               'closing commits linked to PRs']:
        _file_versions_df = get_rows_for_commits(issue, commit_column_name, repo_path)
        file_versions_df = file_versions_df.append(_file_versions_df)
    
    print(file_versions_df.shape)

No python files for 00f6ddb9881a9587c915b3db00fa6e99bd025bf3
No python files for 00f6ddb9881a9587c915b3db00fa6e99bd025bf3
(0, 9)
No python files for 087a6841911119e8e225f3f12541587d98c4f90f
No python files for 087a6841911119e8e225f3f12541587d98c4f90f
(0, 9)
No python files for f7730839ce8beb56339ecd585e07c9f358a15b3c
No python files for f7730839ce8beb56339ecd585e07c9f358a15b3c
No python files for cfec8a780d1b2c345c974a4d0a269c3dff81b76a
(0, 11)


/Users/ilyamezentsev/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


No python files for 098b9707f244e1ad8c3f2f9b94c050fe7c22bb9e
No python files for 098b9707f244e1ad8c3f2f9b94c050fe7c22bb9e
(0, 11)
No python files for 8d48776d426de45513aaa447ee4acdc02218616f
No python files for 8d48776d426de45513aaa447ee4acdc02218616f
(0, 11)
No python files for f10da1957a38bd28b60996ad77fa9f5948a25065
No python files for f10da1957a38bd28b60996ad77fa9f5948a25065
(0, 11)
No python files for dbb88c738ba9aacb161a6a408a38c3f2c1ee6eb0
No python files for dbb88c738ba9aacb161a6a408a38c3f2c1ee6eb0
(0, 11)
No python files for f040b46b01ab8c3ecd182578a5ed277eeaceb45e
(0, 11)
[Errno 2] No such file or directory: '/Users/ilyamezentsev/Documents/zephyr/repos/numpy/numpy/typing/_scalars.py'
(7, 11)
No python files for cba30dbc4012f0968e25d4da3b48f3d9d745aa00
No python files for 547ead6c0c77633df5391cf3523f4f326fc26ea3
No python files for cba30dbc4012f0968e25d4da3b48f3d9d745aa00
No python files for be298a03ba4f35ff67c4fd57f67b7adbf0a32791
No python files for 108555a4ab6637d154686ffe0

/Users/ilyamezentsev/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


No python files for d897435cee1bfcbc9bbcefdae330f349855b7889
(9, 11)
No python files for d558bce8e9d5d4adfb0ab587be20b8a231dd1eea
No python files for d558bce8e9d5d4adfb0ab587be20b8a231dd1eea
No python files for 5cb3283531322f5ad709f1c9a3f618953b2aab32
(9, 11)
(11, 11)
No python files for e1a9b787cd16e714c57a758f353b6eda9cdcee9b
No python files for e1a9b787cd16e714c57a758f353b6eda9cdcee9b
No python files for 64e87207264b9f642b177bcba81701c88f42a79c
(11, 11)
No python files for e1a9b787cd16e714c57a758f353b6eda9cdcee9b
No python files for e1a9b787cd16e714c57a758f353b6eda9cdcee9b
(11, 11)
No python files for b8e0a5cb12bb32c26d8f88af0c6f8eb0e66216d2
No python files for b8e0a5cb12bb32c26d8f88af0c6f8eb0e66216d2
(11, 11)
No python files for 73d8ec888378fc9c8358ba4b88b93f2a48669c31
No python files for 82226d816c67a80ff634fd202c1df32865175232
(11, 11)
No python files for ae0d302c735720739ed4e1bc1e1a231df24356c0
No python files for ae0d302c735720739ed4e1bc1e1a231df24356c0
(11, 11)
No python files

...

invalid syntax (series.py, line 510)
invalid token (test_frame.py, line 2960)
invalid syntax (series.py, line 506)
invalid token (test_frame.py, line 2954)
(27518, 11)
invalid syntax (frame.py, line 3215)
invalid syntax (series.py, line 506)
invalid syntax (frame.py, line 3225)
invalid syntax (series.py, line 506)
[Errno 2] No such file or directory: '/Users/ilyamezentsev/Documents/zephyr/repos/pandas/vb_suite/reshape.py'
(27524, 11)
(27525, 11)
invalid syntax (index.py, line 626)
invalid syntax (series.py, line 506)
Missing parentheses in call to 'print'. Did you mean print(fmonth)? (period.py, line 55)
invalid syntax (index.py, line 624)
invalid syntax (series.py, line 504)
Missing parentheses in call to 'print'. Did you mean print(fmonth)? (period.py, line 55)
(27526, 11)
invalid syntax (series.py, line 505)
invalid syntax (series.py, line 505)
(27526, 11)
invalid token (test_frame.py, line 2838)
invalid token (test_frame.py, line 2838)
invalid token (test_frame.py, line 2832)
inval

(27643, 11)
invalid token (test_frame.py, line 2645)
invalid token (test_frame.py, line 2640)
invalid token (test_frame.py, line 2769)
invalid token (test_frame.py, line 2765)
(27649, 11)
invalid syntax (frame.py, line 3110)
invalid token (test_frame.py, line 2765)
invalid syntax (frame.py, line 3110)
invalid token (test_frame.py, line 2765)
invalid syntax (frame.py, line 3110)
invalid token (test_frame.py, line 2765)
invalid syntax (frame.py, line 3106)
invalid token (test_frame.py, line 2759)
(27649, 11)
(27655, 11)
(27655, 11)
Missing parentheses in call to 'print'. Did you mean print('Warning, ignoring as_index=True')? (groupby.py, line 1203)
invalid syntax (index.py, line 564)
Missing parentheses in call to 'print'. Did you mean print('Warning, ignoring as_index=True')? (groupby.py, line 1203)
invalid syntax (index.py, line 560)
(27660, 11)
No python files for 425ddf8452f4c2554733894f49f1384a9e908157
No python files for 65f2b4327a1ffc40e3fee577ad7d0407ef103769
invalid syntax (fram

(27720, 11)
Missing parentheses in call to 'print'. Did you mean print(i)? (leak.py, line 10)
No python files for 17ecdff7ebe5c4a0d9eac71221ebdecd9422cab5
(27723, 11)
invalid syntax (frame.py, line 2969)
invalid token (test_frame.py, line 2566)
invalid syntax (frame.py, line 2965)
invalid token (test_frame.py, line 2544)
(27725, 11)
No python files for e33309cb63f4b4cee0f9043a22ea07b2e7acca1a
No python files for 439395483158b27bce27549766ed199e25b1d97b
No python files for 5bff8358fd31abb57e8d4203e38a8ab71dd4ff1b
No python files for e7f45cd69d05ed98c261d88bf4a0ea8c5ba30f63
(27726, 11)
No python files for 26545b0d006800a8957437b7fc154aa4aa60aa0e
Missing parentheses in call to 'print'. Did you mean print('Please install python-dateutil via easy_install or some method!')? (datetools.py, line 17)
Missing parentheses in call to 'print'. Did you mean print('Please install python-dateutil via easy_install or some method!')? (datetools.py, line 17)
(27726, 11)
No python files for 0e9b90842e0da2

No python files for a77ac5cebcb3df8c0d98d45bcbe2f0aa5585da3e
(27866, 11)
invalid syntax (frame.py, line 2805)
invalid token (test_frame.py, line 2465)
invalid syntax (frame.py, line 2802)
invalid token (test_frame.py, line 2451)
(27866, 11)
(27866, 11)
invalid token (test_frame.py, line 2404)
invalid token (test_frame.py, line 2400)
(27867, 11)
Missing parentheses in call to 'print'. Did you mean print('Warning, ignoring as_index=True')? (groupby.py, line 801)
Missing parentheses in call to 'print'. Did you mean print('Warning, ignoring as_index=True')? (groupby.py, line 792)
(27870, 11)
No python files for fe16a487a410f796a789eedf1bfceb7c4efa7394
(27874, 11)
invalid syntax (index.py, line 487)
invalid syntax (index.py, line 487)
(27875, 11)
invalid syntax (series.py, line 269)
invalid syntax (series.py, line 269)
(27877, 11)
invalid token (test_frame.py, line 2350)
invalid token (test_frame.py, line 2322)
invalid syntax (series.py, line 284)
invalid syntax (series.py, line 284)
Missin

invalid token (test_frame.py, line 2082)
No python files for 800b70652b98c3bbc05d698654d31db08955d80c
(27979, 11)
Missing parentheses in call to 'print'. Did you mean print(i)? (leak.py, line 10)
(27983, 11)
(27984, 11)
(27986, 11)
Missing parentheses in call to 'print'. Did you mean print('Please install python-dateutil via easy_install or some method!')? (datetools.py, line 17)
Missing parentheses in call to 'print'. Did you mean print('Please install python-dateutil via easy_install or some method!')? (datetools.py, line 17)
(27986, 11)
(27987, 11)
Missing parentheses in call to 'print'. Did you mean print('Warning, ignoring as_index=True')? (groupby.py, line 738)
Missing parentheses in call to 'print'. Did you mean print('Warning, ignoring as_index=True')? (groupby.py, line 741)
(27987, 11)
invalid token (test_frame.py, line 2007)
invalid token (test_frame.py, line 2000)
(27989, 11)
invalid token (test_frame.py, line 2000)
invalid token (test_frame.py, line 2000)
(27989, 11)
(27991

In [49]:
file_versions_df.shape

(27991, 11)

In [51]:
file_versions_df.to_csv('../../data/3_repo_issues_file_versions.tsv', sep='\t', index=False)

In [53]:
file_versions_df.drop(columns=['before_merge', 'after_merge']).head(10)

,commit_sha,commit_type,filename,function_name,issue_url,merge_commit_sha,pr_type,pr_url,py_filename
0,NaN,NaN,numpy/typing/_callable.py,_SignedIntOp.__call__,https://github.com/numpy/numpy/issues/17420,fd01786ea4c7dde540cede258ad11d08d25bacfc,mentioning PRs,https://github.com/numpy/numpy/pull/17429,NaN
0,NaN,NaN,numpy/typing/_callable.py,_IntTrueDiv.__call__,https://github.com/numpy/numpy/issues/17420,fd01786ea4c7dde540cede258ad11d08d25bacfc,mentioning PRs,https://github.com/numpy/numpy/pull/17429,NaN
0,NaN,NaN,numpy/typing/_callable.py,_TD64Div.__call__,https://github.com/numpy/numpy/issues/17420,fd01786ea4c7dde540cede258ad11d08d25bacfc,mentioning PRs,https://github.com/numpy/numpy/pull/17429,NaN
0,NaN,NaN,numpy/typing/_callable.py,_UnsignedIntOp.__call__,https://github.com/numpy/numpy/issues/17420,fd01786ea4c7dde540cede258ad11d08d25bacfc,mentioning PRs,https://github.com/numpy/numpy/pull/17429,NaN
0,NaN,NaN,numpy/typing/_callable.py,_BoolSub.__call__,https://github.com/numpy/numpy/issues/17420,fd01786ea4c7dde540cede258ad11d08d25bacfc,mentioning PRs,https://github.com/numpy/numpy/pull/17429,NaN
0,NaN,NaN,numpy/typing/_callable.py,_BoolTrueDiv.__call__,https://github.com/numpy/numpy/issues/17420,fd01786ea4c7dde540cede258ad11d08d25bacfc,mentioning PRs,https://github.com/numpy/numpy/pull/17429,NaN
0,NaN,NaN,numpy/typing/_callable.py,_FloatOp.__call__,https://github.com/numpy/numpy/issues/17420,fd01786ea4c7dde540cede258ad11d08d25bacfc,mentioning PRs,https://github.com/numpy/numpy/pull/17429,NaN
0,b765975ca89457fea91a63b94075c9e66465e2ea,referencing commits linked to PRs,numpy/core/tests/test_dtype.py,TestPickling.check_pickling,https://github.com/numpy/numpy/issues/16692,NaN,NaN,NaN,NaN
0,2771c2b84a4acbffdf64b17cbc49fab07009722e,referencing commits linked to PRs,numpy/core/tests/test_dtype.py,TestPickling.check_pickling,https://github.com/numpy/numpy/issues/16692,NaN,NaN,NaN,NaN
0,NaN,NaN,sklearn/neighbors/_nearest_centroid.py,NearestCentroid.fit,https://github.com/scikit-learn/scikit-learn/issues/18324,2538489f4ad39d6e32ef4c2fa6297263a26e16ce,mentioning PRs,https://github.com/scikit-learn/scikit-learn/pull/18370,NaN


Сколько issue обработались:

In [54]:
file_versions_df.issue_url.nunique()

3005

Сколько было всего:

In [55]:
issues.url.nunique()

5195